# Data Import
<em>Aaron Wollman, Albin Joseph, Kelsey Richardson Blackwell, Will Huang</em>

In this notebook, the code will import the data for Billboard Top 100, Spotify, and Unemployment from the data folder.  Once the code is imported, it will be cleaned.  After clean-up, all of the data will be merged to be used in the Data Analysis notebook.

In [ ]:
# Dependencies
import pandas as pd

## Import Billboard and Spotify Data
First, the code will import the data from Billboard and Spotify. These are stored in our data/external folder. 

Billboard Top 100 Data is stored in <i>billboard{decade}.csv</i>, while the Spotify data for the Top 100 is stored in <i>billboard{decade}FeaturesDatabase.csv</i>.  

The code loops over each decade and append the dataframes to lists for Billboard and Spotify.

In [ ]:
filepath = "../data/external/billboard_hits/"
def import_billboard_hit(filename):
    columns = ["Date"]
    columns.extend(range(1,101))
    return pd.read_csv(f"{filepath}{filename}.csv", \
                       encoding="ISO-8859-1", names=columns)

def import_billboard_feature(filename):
    return pd.read_csv(f"{filepath}{filename}.csv", \
                       encoding="ISO-8859-1")

billboard_hits = []
billboard_features = []
for year in range(1960, 2020, 10):
    print(f"Importing {year}'s top 100.")
    billboard_hit = import_billboard_hit(f"billboard{year}")
    print(f"Importing {year}'s features.")
    billboard_feature = import_billboard_feature(f"billboard{year}FeaturesDatabase")
    
    billboard_hits.append(billboard_hit)
    billboard_features.append(billboard_feature)

In [ ]:
billboard_hits[0].head()

In [ ]:
billboard_features[0].head()

### Combine the Decade Data for Billboard and Spotify
Now that the code has the data as a list of dataframes per decade for both the Top 100 and the Spotify data, the code now needs to combine the decade data for easier processing.  There will be two dataframes with the Top 100 and the Spotify data, respectively.

In [ ]:
# Combine Hits Dataframes
all_billboard_hits = pd.concat(billboard_hits)
all_billboard_hits = all_billboard_hits.reset_index()
all_billboard_hits["Date"].nunique()

In [ ]:
# Combine Features Dataframes
all_billboard_features = pd.concat(billboard_features)
all_billboard_features["Track"].nunique()

### Data Cleanup
This data will be merged with the unemployment data later on the year and month. To be able to do so, the code needs to split the Date column in the Billboard data into Year, Month, and Day.

In [ ]:
# Split the date
split_dates = all_billboard_hits["Date"].str.split(pat = "-", expand=True)
split_dates.columns = ["Year", "Month", "Day"]
all_billboard_hits["Year"] = split_dates["Year"]
all_billboard_hits["Month"] = split_dates["Month"]
all_billboard_hits["Day"] = split_dates["Day"]
all_billboard_hits.head()

Then, to be able to merge the Spotify and Billboard dataframes, the code needs to split the actual song and artist data as those are not discrete at this point.

In [ ]:
# Reform the hits table to split up song/artist to be able to merge
def split_song_artist(song_artist):
    song_artist = song_artist.replace('"', "'")
    split = song_artist.split("', '")
    song = split[0].replace("['", "")
    artist = split[1].replace("']", "")
    return (song, artist)

In [ ]:
song_datas = []
for index, row in all_billboard_hits.iterrows():
    for hit_num in range(1, 101):
        song_artist = row[hit_num]
        try:
            (song, artist) = split_song_artist(song_artist)
        except:
            print(f"Couldn't parse {song_artist}")
            song = ""
            artist = ""
        song_data = {
            "Track" : song,
            "Artist" : artist,
            "Placement" : hit_num,
            "Year" : row["Year"],
            "Month" : row["Month"],
            "Day" : row["Day"],
        }
        song_datas.append(song_data)

In [ ]:
all_hits = pd.DataFrame(song_datas)
all_hits.head()

### Merge Billboard and Spotify Data
Now that the data is cleaned up, the Billboard and Spotify data can be merged on the song title and artist.  This merged dataframe will be used later to be merged with the unemployment data.

In [ ]:
# Merge Billboard Data and Spotify Data
final_music = pd.merge(all_hits, all_billboard_features, on=["Track", "Artist"])
columns_to_drop = ["URI", "mode", "duration_ms", "instrumentalness", 
                   "time_signature", "chorusHit", "sections"]
final_music["Year"] = final_music["Year"].astype("int")
final_music = final_music.drop(columns = columns_to_drop)
final_music = final_music.dropna()
final_music.head()

In [ ]:
# Save to CSV
final_music.to_csv("../data/billboard_hits.csv")

## Import Unemployment

In [ ]:
umemployment_data = pd.read_csv("../data/external/unemployment_rate.csv")
umemployment_data.head()

After import, the unemployment data isn't structured for merging with the music data. The issue is that the dataframe has the months as columns. Months needs to be a column for merging, so the data needs to be flattened.

In [ ]:
unemployment_months = []
for index, row in umemployment_data.iterrows():
    year = row["Year"]
    for month_num in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]:
        unemployment_rate = row[month_num]
        unemployment_month = {
            "Year" : year,
            "Month" : month_num,
            "Unemployment Rate" : unemployment_rate
        }        
        unemployment_months.append(unemployment_month)

unemployment_months[0]

In [ ]:
unemployment_df = pd.DataFrame(unemployment_months)
unemployment_df["Year"] = unemployment_df["Year"].astype("int")
unemployment_df.head()

In [ ]:
unemployment_df.to_csv("../data/unemployment.csv")

## Merge Music and Unemployment
Now that the code has the music and unemployment data in a format that can be merged, it's time to do the merge.  For each song, the code will add the unemployment data for the year and months when the song was on the Top 100.  This data will then be used in the Data Analysis notebook.

In [ ]:
music_and_unemployment = pd.merge(final_music, unemployment_df,\
                                  on=["Year", "Month"])
music_and_unemployment.head()

In [ ]:
music_and_unemployment.to_csv("../data/music_and_unemployment.csv")